In [1]:
import os
import json
import shutil
from PIL import Image, ImageDraw
from pathlib import Path
import natsort

# -----------------------------
# 설정 섹션
# -----------------------------

# 기본 디렉토리 설정
base_dir = os.getcwd()

# 폴더 경로 설정
json_dir = os.path.join(base_dir, "JSON")
images_dir = os.path.join(base_dir, "images")
json_temp_dir = os.path.join(base_dir, "JSON_temp")
txt_c7_dir = os.path.join(base_dir, "TXT_C7")
images_c7_resize_dir = os.path.join(base_dir, "images_C7_resize")
images_c7_bbox_dir = os.path.join(base_dir, "images_C7_BBOX")

# 필요한 폴더 생성
os.makedirs(json_temp_dir, exist_ok=True)
os.makedirs(txt_c7_dir, exist_ok=True)
os.makedirs(images_c7_resize_dir, exist_ok=True)
os.makedirs(images_c7_bbox_dir, exist_ok=True)

# 지원하는 이미지 확장자 목록
image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}

# -----------------------------
# 함수 정의 섹션
# -----------------------------

def copy_json_files():
    """
    images 폴더에 있는 이미지 파일과 이름이 동일한 .json 파일을 JSON_temp 폴더로 복사.
    """
    image_basenames = {os.path.splitext(f)[0] for f in os.listdir(images_dir)
                       if os.path.isfile(os.path.join(images_dir, f)) and os.path.splitext(f)[1].lower() in image_extensions}

    json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]
    copied_files = 0

    for json_file in json_files:
        base_name = os.path.splitext(json_file)[0]
        if base_name in image_basenames:
            shutil.copy(os.path.join(json_dir, json_file), os.path.join(json_temp_dir, json_file))
            copied_files += 1

    print(f"JSON_temp 폴더로 복사된 JSON 파일 개수: {copied_files}")

def compress_image_to_jpg(input_path, output_path, quality=85):
    """
    이미지 압축 및 JPG 형식 변환 함수
    """
    try:
        with Image.open(input_path) as img:
            output_path = os.path.splitext(output_path)[0] + ".jpg"
            rgb_img = img.convert("RGB")
            rgb_img.save(output_path, "JPEG", optimize=True, quality=quality)
        return True
    except Exception as e:
        print(f"Error compressing {input_path}: {e}")
        return False

def convert_json_to_yolo(json_file):
    """
    JSON_temp 폴더에 있는 .json 파일을 YOLO v5 형식의 .txt 파일로 변환하고,
    해당 이미지 파일을 images_C7_resize 폴더로 압축 복사.
    """
    json_path = os.path.join(json_temp_dir, json_file)
    txt_filename = os.path.splitext(json_file)[0] + ".txt"
    output_txt_path = os.path.join(txt_c7_dir, txt_filename)

    try:
        with open(json_path, "r", encoding="utf-8-sig") as file:
            data = json.load(file)
    except Exception as e:
        print(f"JSON 파일 읽기 오류: {json_file}, 오류: {e}")
        return False

    annotations = data.get("annotations", [])
    images = {img["id"]: img for img in data.get("images", [])}

    yolo_annotations = []
    associated_image_files = set()

    for annotation in annotations:
        if annotation.get("category_id") != 7:
            continue

        bbox = annotation.get("bbox", [])
        if not bbox or len(bbox) != 4:
            print(f"잘못된 bbox 형식: {bbox} in {json_file}")
            continue

        image_id = annotation.get("image_id")
        image_info = images.get(image_id, {})
        image_file_name = image_info.get("file_name", "")
        image_path = os.path.join(images_dir, image_file_name)

        if not os.path.exists(image_path):
            print(f"이미지 파일 없음: {image_file_name} (JSON 파일: {json_file})")
            continue

        try:
            with Image.open(image_path) as img:
                image_width, image_height = img.size
        except Exception as e:
            print(f"이미지 파일 읽기 오류: {image_file_name}, 오류: {e}")
            continue

        x_min, y_min, x_max, y_max = bbox
        x_center = ((x_min + x_max) / 2) / image_width
        y_center = ((y_min + y_max) / 2) / image_height
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height

        yolo_annotations.append(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
        associated_image_files.add(image_file_name)

    if yolo_annotations:
        with open(output_txt_path, "w") as file:
            file.write("\n".join(yolo_annotations) + "\n")

        for image_file_name in associated_image_files:
            src_image_path = os.path.join(images_dir, image_file_name)
            dst_image_path = os.path.join(images_c7_resize_dir, image_file_name)
            compress_image_to_jpg(src_image_path, dst_image_path)
        return True
    return False

def draw_bounding_boxes(image_path, label_path, output_path):
    """
    이미지에 바운딩 박스를 그려 결과를 저장.
    """
    try:
        with open(label_path, "r") as f:
            labels = [line.strip() for line in f.readlines()]

        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        width, height = image.size

        for label in labels:
            data = label.split(" ")
            if len(data) != 5:
                print(f"잘못된 라벨 형식: {label_path}")
                continue

            c, x, y, w, h = map(float, data)
            x_min = (x - w / 2) * width
            y_min = (y - h / 2) * height
            x_max = (x + w / 2) * width
            y_max = (y + h / 2) * height

            draw.rectangle([x_min, y_min, x_max, y_max], outline="blue", width=3)

        image.save(output_path)
        print(f"바운딩 박스가 그려진 이미지를 저장했습니다: {output_path}")

    except Exception as e:
        print(f"오류 발생: {e}, 파일: {image_path}")

def main():
    # Step 1: JSON 파일을 YOLO 형식으로 변환하고 이미지 압축
    copy_json_files()
    json_files = [f for f in os.listdir(json_temp_dir) if f.endswith(".json")]
    for json_file in json_files:
        convert_json_to_yolo(json_file)

    # Step 2: BBOX 그리기
    image_files = natsort.natsorted([
        os.path.join(images_c7_resize_dir, f) for f in os.listdir(images_c7_resize_dir) if f.lower().endswith(tuple(image_extensions))
    ])
    label_files = natsort.natsorted([
        os.path.join(txt_c7_dir, f) for f in os.listdir(txt_c7_dir) if f.endswith('.txt')
    ])

    if len(image_files) != len(label_files):
        print("이미지 파일과 라벨 파일의 개수가 일치하지 않습니다!")
    else:
        for img, lbl in zip(image_files, label_files):
            filename = Path(img).stem
            output_extension = Path(img).suffix
            output_path = os.path.join(images_c7_bbox_dir, f"{filename}_bbox{output_extension}")
            draw_bounding_boxes(img, lbl, output_path)

if __name__ == "__main__":
    main()


JSON_temp 폴더로 복사된 JSON 파일 개수: 63
바운딩 박스가 그려진 이미지를 저장했습니다: h:\Vehicle_3_Rtest\images_C7_BBOX\공항로_차량_20220916_0944_5m_15도_4085_CLOUDY_000117_bbox.jpg
바운딩 박스가 그려진 이미지를 저장했습니다: h:\Vehicle_3_Rtest\images_C7_BBOX\공항로_차량_20220916_0944_5m_15도_4085_CLOUDY_000291_bbox.jpg
바운딩 박스가 그려진 이미지를 저장했습니다: h:\Vehicle_3_Rtest\images_C7_BBOX\공항로_차량_20220916_0944_5m_15도_4085_CLOUDY_000292_bbox.jpg
바운딩 박스가 그려진 이미지를 저장했습니다: h:\Vehicle_3_Rtest\images_C7_BBOX\공항로_차량_20220916_0944_5m_15도_4085_CLOUDY_000293_bbox.jpg
바운딩 박스가 그려진 이미지를 저장했습니다: h:\Vehicle_3_Rtest\images_C7_BBOX\공항로_차량_20220916_0944_5m_15도_4085_CLOUDY_000294_bbox.jpg
바운딩 박스가 그려진 이미지를 저장했습니다: h:\Vehicle_3_Rtest\images_C7_BBOX\공항로_차량_20220916_0944_5m_15도_4085_CLOUDY_000295_bbox.jpg
바운딩 박스가 그려진 이미지를 저장했습니다: h:\Vehicle_3_Rtest\images_C7_BBOX\공항로_차량_20220916_0944_5m_15도_4085_CLOUDY_000296_bbox.jpg
바운딩 박스가 그려진 이미지를 저장했습니다: h:\Vehicle_3_Rtest\images_C7_BBOX\공항로_차량_20220916_0944_5m_15도_4086_CLOUDY_000087_bbox.jpg
바운딩 박스가 그려진 이미지를 저장했습니다: h:\Vehicle_3_Rtest\ima